# IBM Data Science Capstone Project
This is a part of IBM's 9 course specialization <a href="https://www.coursera.org/professional-certificates/ibm-data-science">avalible on Coursera</a>

## Setup:
We have to import some major libraries that will make the project shine!

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


#### Author:
Ilya Smelyanskiy
<a href="https://www.linkedin.com/in/ilya-smelyanskiy/">LinkedIn</a>